In [ ]:
import cv2
from google.colab.patches import cv2_imshow  # Required for Google Colab
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from pathlib import Path

# Function to detect objects using YOLOv5
def detect_objects_yolov5(frame, model):
    results = model(frame)
    return results

# Function to process the webcam feed
def process_webcam():
    # JavaScript for capturing photo from webcam
    js = Javascript('''
        async function takePhoto(quality) {
          const div = document.createElement('div');
          const capture = document.createElement('button');
          capture.textContent = 'Capture';
          div.appendChild(capture);

          const video = document.createElement('video');
          video.style.display = 'block';
          const stream = await navigator.mediaDevices.getUserMedia({video: true});

          document.body.appendChild(div);
          div.appendChild(video);
          video.srcObject = stream;
          await video.play();

          // Resize the output to fit the video element.
          google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

          // Wait for Capture to be clicked.
          await new Promise((resolve) => capture.onclick = resolve);

          const canvas = document.createElement('canvas');
          canvas.width = video.videoWidth;
          canvas.height = video.videoHeight;
          canvas.getContext('2d').drawImage(video, 0, 0);
          stream.getVideoTracks()[0].stop();
          div.remove();
          return canvas.toDataURL('image/jpeg', quality);
        }
        ''')

    # Display JavaScript for capturing photo from webcam
    display(js)

    # Function to capture photo from webcam
    def take_photo(filename='photo.jpg', quality=0.8):
        data = eval_js('takePhoto({})'.format(quality))
        binary = b64decode(data.split(',')[1])
        with open(filename, 'wb') as f:
            f.write(binary)
        return filename

    # Capture photo from webcam
    img_path = take_photo()

    # Load YOLOv5 model
    import torch
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

    # Perform object detection on captured image
    img = cv2.imread(img_path)
    results = detect_objects_yolov5(img, model)

    # Display results
    results.show()

# Process webcam feed and perform object detection
process_webcam()
